In [0]:
!pip install tensorflow==2.0.0

## 모델 불러오기

In [0]:
from tensorflow.keras.applications.vgg16 import VGG16

# 기존의 1000 클래스의 출력을 사용하지 않기 때문에
# include_top=False 로 출력층을 포함하지 않은 상태로 불러들임
vgg16 = VGG16(include_top=False, input_shape=(224,224,3))

In [0]:
# 출력층이 포함되지 않은 것을 확인
vgg16.summary()

## 모델편집

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

# 모델을 편집하기 위해 함수를 사용
def build_transfer_model(vgg16):

  # 호출한 모델을 사용해서 새로운 모델 작성
  model = Sequential(vgg16.layers)

  # 호출한 가중치의 일부는 재학습하지 않도록 설정
  # 추가한 층과 출력층에 가까운 층의 가중치만 재학습
  for layer in model.layers[:15]:
    layer.trainable = False
  
  # 출력할 추가부분의 층을 구축
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  return model

In [0]:
# 정의한 함수를 호출하여 모델을 생성
model = build_transfer_model(vgg16)

In [0]:
# 완성된 model
model.summary()

## 모델 컴파일

In [0]:
from tensorflow.keras.optimizers import SGD

# 기존의 VGG16의 학습된 가중치를 활용하면서 현재의 문제에도 가중치를 적용하기 위해
# 옵티마이저의 학습률을 낮게(1e-4) 설정
# 한번에 가중치를 크게 수정하지 않는 것이 포인트
model.compile(loss='binary_crossentropy', 
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

## 입력 이미지 준비

In [0]:
# Colab 에서 로컬 파일 업로드
from google.colab import files
uploaded = files.upload()

In [0]:
!ls

In [0]:
from tensorflow.keras.preprocessing.image import load_img

# 이미지 불러오기. load_img() 에서 이미지 리사이징 가능
# VGG16의 입력 사이즈인 224x224 로 리사이즈
img_dog = load_img('dog.jpg', target_size=(224, 224))
img_cat = load_img('cat.jpg', target_size=(224, 224))

In [0]:
type(img_dog)

In [0]:
# Pillow 형태의 이미지를 numpy array 로 변환
from tensorflow.keras.preprocessing.image import img_to_array

arr_dog = img_to_array(img_dog)
arr_cat = img_to_array(img_cat)

In [0]:
type(arr_dog), arr_dog.shape

In [0]:
# 이미지를 VGG16이 학습할 때와 같이 변환
from tensorflow.keras.applications.vgg16 import preprocess_input

arr_dog = preprocess_input(arr_dog)
arr_cat = preprocess_input(arr_cat)

In [0]:
type(arr_dog), arr_dog.shape

In [0]:
# 일반적으로 딥러닝 모델에서는 여러장의 사진을 일괄적으로 입력하고 배열로 결과를 받음
# 2장의 이미지를 배열로 만듬

import numpy as np

arr_input = np.stack([arr_dog, arr_cat])

In [0]:
arr_input.shape

## 예측

In [0]:
# 예측값 산출
# 추론에서는 2x1000의 2차원 배열이 출력됨
prediction = model.predict(arr_input)

In [0]:
# 예측값 shape 확인
prediction.shape

In [0]:
prediction

In [0]:
# 이미지의 예측결과 확인
# 1000개의 카테고리 중에서 가장 확률이 높은 상위 5개를 표시

from tensorflow.keras.applications.vgg16 import decode_predictions
result = decode_predictions(prediction)

In [0]:
result